In [1]:
eicu_path = '/scratch/wiensj_root/wiensj/shared_data/datasets/eicu-2.0/'
save_path = '/scratch/wiensj_root/wiensj/shared_data/FIDDLE_project/extracted/'

import pandas as pd
import numpy as np
from tqdm import tqdm
pd.options.display.max_columns = 100

## RespiratoryCare

In [3]:
df_R = pd.read_csv(eicu_path + '{}.csv'.format('respiratoryCare'))

/sw/arcts/centos7/python3.7-anaconda/2019.07/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4,5,6,12,26,27,28,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
for c in ['respcarestatusoffset', 'ventstartoffset', 
          'ventendoffset', 'priorventstartoffset', 
          'priorventendoffset']:
    df_R[c] = df_R[c].replace({0: np.nan})

In [5]:
df_R.columns

Index(['respcareid', 'patientunitstayid', 'respcarestatusoffset',
       'currenthistoryseqnum', 'airwaytype', 'airwaysize', 'airwayposition',
       'cuffpressure', 'ventstartoffset', 'ventendoffset',
       'priorventstartoffset', 'priorventendoffset', 'apneaparms',
       'lowexhmvlimit', 'hiexhmvlimit', 'lowexhtvlimit', 'hipeakpreslimit',
       'lowpeakpreslimit', 'hirespratelimit', 'lowrespratelimit',
       'sighpreslimit', 'lowironoxlimit', 'highironoxlimit',
       'meanairwaypreslimit', 'peeplimit', 'cpaplimit', 'setapneainterval',
       'setapneatv', 'setapneaippeephigh', 'setapnearr', 'setapneapeakflow',
       'setapneainsptime', 'setapneaie', 'setapneafio2'],
      dtype='object')

In [16]:
df_R1 = df_R.drop(columns=[
    'respcareid', 'currenthistoryseqnum', 
    'ventendoffset', 
    'priorventendoffset', 
])
df_R1 = df_R1.rename(columns={
    'patientunitstayid': 'ID',
    'respcarestatusoffset': 't',
})
df_R1 = df_R1.set_index(['ID', 't'])
df_R1.columns.name = 'variable_name'
df_R1 = df_R1.stack()
df_R1.name = 'variable_value'
df_R1 = df_R1.reset_index()

In [30]:
# need to make sure ventStartOffset is before the recording time
df_R1[df_R1['variable_name'] == 'ventstartoffset'][['t', 'variable_value']] \
    .apply(lambda x: x[0] >= x[1], axis=1).mean()

# remove rows with ventStartOffset before the recording time
df_R2 = df_R1.loc[
    df_R1.apply(lambda x: x[2] != 'ventstartoffset' or (x[2] == 'ventstartoffset' and x[1] >= x[3]), axis=1)
]

In [33]:
df_R2.to_pickle(save_path + '{}.pickle'.format('respiratoryCare'))

## IntakeOutput

In [2]:
df_IO = pd.read_csv(eicu_path + '{}.csv'.format('intakeOutput'))

In [3]:
df_IO = df_IO.drop(columns=['intakeoutputid', 'intakeoutputentryoffset', 'celllabel']) \
    .rename(columns={
    'patientunitstayid': 'ID', 'intakeoutputoffset': 't', 
})

In [4]:
df_IO1 = df_IO[['ID', 't', 'intaketotal', 'outputtotal', 'dialysistotal', 'nettotal']]
df_IO1 = df_IO1.set_index(['ID', 't'])
df_IO1.columns.name = 'variable_name'
df_IO1 = df_IO1.stack()
df_IO1.name = 'variable_value'
df_IO1 = df_IO1.reset_index()

In [5]:
df_IO2 = df_IO[['ID', 't', 'cellpath', 'cellvaluenumeric', 'cellvaluetext']] \
    .rename(columns={'cellpath': 'variable_name'})
df_IO2['variable_value'] = pd.to_numeric(df_IO2['cellvaluetext'], errors='ignore')
df_IO2 = df_IO2[['ID', 't', 'variable_name', 'variable_value']]

In [6]:
del df_IO
df_IOo = pd.concat([df_IO1, df_IO2])

In [7]:
df_IOo.to_parquet(save_path + 'intakeOutput.parquet')